In [1]:
from logger import Logger
logger = Logger()
logger.info("テスト")

テスト


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
         else "cpu")
logger.info(f"Using {device} device")

Using mps device


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

In [12]:
y_pred

tensor([9], device='mps:0')

In [13]:
input_image = torch.rand(3, 28, 28)
input_image.size()

torch.Size([3, 28, 28])

In [14]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

In [15]:
type(flatten)

torch.nn.modules.flatten.Flatten

In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

In [17]:
hidden1

tensor([[ 0.6405,  0.1614, -0.4434,  0.5445,  0.1476,  0.1577, -0.0958, -0.2241,
          0.5607,  0.2799, -0.2294,  0.2297,  0.1822,  0.1473,  0.0187,  0.1210,
          0.5212, -0.4547, -0.1337, -0.3730],
        [ 0.6550, -0.0275, -0.0849,  0.2656,  0.0714,  0.6719, -0.2748, -0.2002,
          0.1538,  0.2556, -0.2833,  0.3154, -0.1174, -0.2762,  0.2165, -0.0455,
          0.5389, -0.0032, -0.0888, -0.0425],
        [ 0.4504, -0.1871, -0.5958,  0.3419,  0.3230,  0.2191, -0.0319, -0.1561,
          0.3751, -0.0271, -0.2851,  0.2385,  0.0471, -0.1713,  0.0120,  0.0883,
          0.1291, -0.2181, -0.0183, -0.6466]], grad_fn=<AddmmBackward0>)

In [18]:
hidden1 = nn.ReLU()(hidden1)

In [19]:
hidden1

tensor([[0.6405, 0.1614, 0.0000, 0.5445, 0.1476, 0.1577, 0.0000, 0.0000, 0.5607,
         0.2799, 0.0000, 0.2297, 0.1822, 0.1473, 0.0187, 0.1210, 0.5212, 0.0000,
         0.0000, 0.0000],
        [0.6550, 0.0000, 0.0000, 0.2656, 0.0714, 0.6719, 0.0000, 0.0000, 0.1538,
         0.2556, 0.0000, 0.3154, 0.0000, 0.0000, 0.2165, 0.0000, 0.5389, 0.0000,
         0.0000, 0.0000],
        [0.4504, 0.0000, 0.0000, 0.3419, 0.3230, 0.2191, 0.0000, 0.0000, 0.3751,
         0.0000, 0.0000, 0.2385, 0.0471, 0.0000, 0.0120, 0.0883, 0.1291, 0.0000,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10),
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [22]:
logits

tensor([[ 0.0576, -0.0468, -0.0059,  0.3833,  0.4794, -0.1374,  0.0778,  0.1474,
         -0.0531, -0.0179],
        [ 0.2006, -0.2238, -0.1372,  0.3944,  0.3680,  0.0742,  0.1351,  0.1222,
          0.0540,  0.1790],
        [ 0.1038, -0.0327, -0.0974,  0.3167,  0.4804, -0.1580, -0.0581,  0.1070,
         -0.1144,  0.0172]], grad_fn=<AddmmBackward0>)

In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [24]:
pred_probab

tensor([[0.0952, 0.0857, 0.0893, 0.1318, 0.1451, 0.0783, 0.0971, 0.1041, 0.0852,
         0.0882],
        [0.1070, 0.0700, 0.0763, 0.1298, 0.1265, 0.0943, 0.1002, 0.0989, 0.0924,
         0.1047],
        [0.1028, 0.0897, 0.0841, 0.1272, 0.1498, 0.0791, 0.0874, 0.1031, 0.0826,
         0.0943]], grad_fn=<SoftmaxBackward0>)